In [6]:
import os
import glob
import datetime as dt
import numpy as np
import pandas as pd
from sqlalchemy import text
from trading.db import db_helper_binance

engine = db_helper_binance.engine

In [7]:
symbol = "AAVEUSDT".lower()
h4, h1, m15 = "4h", "1h", "15m"
timeframes = (h4, h1, m15)

lips, teeth, jaw = 5, 13, 34
periods = (lips, teeth, jaw)

file_patterns = ("merged_data", "filtered_data", "intervals")

df_all = {}

In [8]:
# Определение Функций

# Загружаем данные из БД
def load_data(
        symbol: str,
        timeframe: str,
        **kwargs,
) -> pd.DataFrame:
    table_kline = f'klines_{symbol}_{timeframe}'
    # table_ema = f'klines_{symbol}_{timeframe}_ema'
    # Создаем SQL-запрос для загрузки данных из обеих таблиц
    # query = text(f"""
    #     SELECT 
    #         k.time_open,
    #         k.open,
    #         k.high,
    #         k.low,
    #         k.close,
    #         k.volume,
    #         e.ma_{lips},
    #         e.ma_{teeth},
    #         e.ma_{jaw},
    #         e.ma_{lips}_delta,
    #         e.ma_{teeth}_delta,
    #         e.ma_{jaw}_delta,
    #     FROM 
    #         {table_kline} k
    #     JOIN 
    #         {table_ema} e ON k.time_open = e.time_open
    #     ORDER BY 
    #         k.time_open
    # """)
    query = text(f"""
        SELECT 
            time_open,
            open,
            high,
            low,
            close
        FROM
            {table_kline}
        ORDER BY 
            time_open
    """)
    
    # Выполняем запрос и загружаем данные в DataFrame
    df = pd.read_sql_query(query, engine)
    
    # Добавление пустых столбцов
    # for column in lips, teeth, jaw:
    #     new_column = f'{timeframe}_ma_{column}'
    #     df = df.assign(**{new_column:np.nan})
    #     new_column = f'{timeframe}_ma_{column}_delta'
    #     df = df.assign(**{new_column:np.nan})
    
    return df

def check_integrity_and_fill_gaps(
        df: pd.DataFrame,
        interval_name: str,
) -> pd.DataFrame:
    interval = pd.Timedelta(interval_name)
    
    # Преобразуем 'time_open' в datetime, если это еще не сделано
    df['time_open'] = pd.to_datetime(df['time_open'], unit='ms')
    
    # Сортируем DataFrame по времени, чтобы убедиться, что данные в правильном порядке
    df = df.sort_values('time_open')
    
    # Проверяем интервалы между строками
    time_diff = df['time_open'].diff()
    expected_diff = interval  # Ожидаемый интервал 15 минут
    
    # Находим пропуски
    missing_intervals = df[time_diff > expected_diff]
    
    if not missing_intervals.empty:
        print(f"Найдено {len(missing_intervals)} пропущенных {interval} интервалов.")
        # list(map(lambda x: print(x), missing_intervals['time_open'].tolist()))
    
        # Создаем полный диапазон дат с шагом в 1 час
        full_range = pd.date_range(start=df['time_open'].min(), end=df['time_open'].max(), freq=interval)
    
        # Создаем новый DataFrame с полным диапазоном дат
        df_full = pd.DataFrame({'time_open': full_range})
    
        # Объединяем с исходным DataFrame
        df = pd.merge(df_full, df, on='time_open', how='left')
    
        # Заполняем пропуски средними значениями
        df = df.interpolate(method='linear')
    
        print(interval_name, "Пропуски заполнены.")
    else:
        print(interval_name, "Пропусков в данных не обнаружено.")
        
    print(interval_name, "\nStart date:", df['time_open'].min(), "\nEnd date:", df['time_open'].max())
    # print(df.tail(3))
    
    return df

# Функция для расчета EMA
def calculate_ema(quote, previous_ma, period):
    k = 2 / (period + 1)
    return quote * k + previous_ma * (1 - k)

def calculate_emas(df, periods):
    for period in periods:
        column_name = f'ma_{period}'
        
        # Используем первое значение 'open' как начальное значение EMA
        # df[column_name] = df['open'].copy()
        df.loc[df.index[0], column_name] = previous_ma = df['open'].iloc[0]
        
        for i in range(1, len(df)):
            previous_ma = calculate_ema(
                df['open'].iloc[i], 
                previous_ma, 
                period,
            )
            df.loc[df.index[i], column_name] = previous_ma
            df.loc[df.index[i], f'{column_name}_delta'] = (
                (previous_ma / df.loc[df.index[i-1], column_name] - 1) * 100
            )
    
    return df

# Функция для заполнения NaN значений
def fill_nan_ema(df, column, period, timeframe):
    
    time_open_as_list = tuple(df['time_open'].to_list())
    for index, row in df.iterrows():
        if pd.isna(row[column]):
            if timeframe == '1h':
                # вычисляем начало периода для заполнения промежуточных значений EMA
                period_start = row['time_open'].replace(minute=0, second=0, microsecond=0) - pd.Timedelta(hours=1)
            elif timeframe == '4h':
                # вычисляем начало периода для заполнения промежуточных значений EMA
                period_start = row['time_open'].replace(
                    hour=row['time_open'].hour - row['time_open'].hour % 4, 
                    minute=0, 
                    second=0, 
                    microsecond=0
                ) - pd.Timedelta(hours=4)
            
            if period_start not in time_open_as_list:
                print(f'Нет периода для расчёта EMA: {period_start}\nperiod: {period}\ntimeframe: {timeframe}\n', '='*20, '\n')
                continue
            period_start_index = df.index[df['time_open'] == period_start][0]          
            previous_ma = df.loc[period_start_index, column]
            ema = calculate_ema(row[f'{m15}_open'], previous_ma, period)
            ema_delta = (ema / df.loc[period_start_index, column] - 1) * 100
            df.at[index, column] = ema
            df.at[index, f'{column}_delta'] = ema_delta
    
    return df


def save_df_to_excel(df: pd.DataFrame, file_name: str, patterns: tuple = file_patterns) -> None:
    if file_name not in patterns:
        print(f'{file_name} - неправильное имя')
        return
    # Сохраняем отфильтрованный результат в новый Excel файл
    dir_file = os.path.expanduser("~/Downloads")
    file = os.path.join(dir_file, f'{file_name}__{dt.datetime.now(dt.UTC).strftime('%Y_%m_%d_%H_%M_%S.%f')}.xlsx')
    df.to_excel(file, index=False, engine='openpyxl')
    print(f"\nИнтервалы данных сохранены в файл: {file}")
    
    
def load_data_from_excel(file_name: str, patterns: tuple = file_patterns) -> pd.DataFrame | None:    
    # Загрузка данных из Excel файла для анализа
    if file_name not in patterns:
        # file_patterns = ("merged_data", "filtered_data", "intervals")
        print(f'{file_name} - неправильное имя')
        return

    # Путь к директории с файлами
    directory = os.path.expanduser("~/Downloads")
    
    # Шаблон имени файла
    file_pattern = f"{file_name}__*.xlsx"
    
    # Получаем список всех файлов, соответствующих шаблону
    files = glob.glob(os.path.join(directory, file_pattern))
    
    if not files:
        print("Не найдено файлов, соответствующих шаблону.")
    else:
        # Сортируем файлы по дате изменения (самый новый - первый)
        latest_file = max(files, key=os.path.getmtime)
    
        # Загружаем данные из файла
        df = pd.read_excel(latest_file, engine='openpyxl')
    
        # Выводим информацию о загруженных данных
        print(f"Загружено строк: {len(df)}")
        # print("\nПервые несколько строк данных:")
        # print(df_merged.head())

In [4]:
# Load, check data integrity and fill gaps 
for timeframe in timeframes:
    df = load_data(symbol, timeframe)
    df = check_integrity_and_fill_gaps(df, timeframe)
    df = calculate_emas(df, periods)
    df = df.add_prefix(f'{timeframe}_')
    df = df.rename(columns={f'{timeframe}_time_open': 'time_open'})
    df_all[timeframe] = df
    

Найдено 1 пропущенных 0 days 04:00:00 интервалов.
2024-11-20 00:00:00
4h Пропуски заполнены.
4h 
Start date: 2020-10-15 00:00:00 
End date: 2024-12-24 20:00:00
Найдено 1 пропущенных 0 days 01:00:00 интервалов.
2024-11-20 00:00:00
1h Пропуски заполнены.
1h 
Start date: 2020-10-15 03:00:00 
End date: 2024-12-24 23:00:00
Найдено 1 пропущенных 0 days 00:15:00 интервалов.
2024-11-20 00:00:00
15m Пропуски заполнены.
15m 
Start date: 2020-10-15 03:00:00 
End date: 2024-12-24 23:45:00


In [5]:
# Выполняем слияние всех интервалов в один на основе time_open
df_merged = pd.merge(df_all['15m'], df_all['1h'], on='time_open', how='left')
df_merged = pd.merge(df_merged, df_all['4h'], on='time_open', how='left')

In [6]:
# ДОЛГО
# Заполняем промежуточные NaN значения для 1h и 4h расчитывая промежуточные EMA для каждого периода 
for tf in ['1h', '4h']:
    for period in periods:
        df_merged = fill_nan_ema(df_merged, f'{tf}_ma_{period}', period, tf)

save_df_to_excel(df_merged, 'merged_data')

Нет периода для расчёта EMA: 2020-10-15 02:00:00
period: 5
timeframe: 1h

Нет периода для расчёта EMA: 2020-10-15 02:00:00
period: 5
timeframe: 1h

Нет периода для расчёта EMA: 2020-10-15 02:00:00
period: 5
timeframe: 1h

Нет периода для расчёта EMA: 2020-10-15 02:00:00
period: 13
timeframe: 1h

Нет периода для расчёта EMA: 2020-10-15 02:00:00
period: 13
timeframe: 1h

Нет периода для расчёта EMA: 2020-10-15 02:00:00
period: 13
timeframe: 1h

Нет периода для расчёта EMA: 2020-10-15 02:00:00
period: 34
timeframe: 1h

Нет периода для расчёта EMA: 2020-10-15 02:00:00
period: 34
timeframe: 1h

Нет периода для расчёта EMA: 2020-10-15 02:00:00
period: 34
timeframe: 1h

Нет периода для расчёта EMA: 2020-10-14 20:00:00
period: 5
timeframe: 4h

Нет периода для расчёта EMA: 2020-10-14 20:00:00
period: 5
timeframe: 4h

Нет периода для расчёта EMA: 2020-10-14 20:00:00
period: 5
timeframe: 4h

Нет периода для расчёта EMA: 2020-10-14 20:00:00
period: 5
timeframe: 4h

Нет периода для расчёта EMA: 202

In [1]:
# Поиск ВХОДА ВАРИАНТ 1
#
# Фильтруем датафрейм по указанным условиям и сохраняем
filtered_df = df_merged[
    (df_merged[f'4h_ma_{lips}'] > df_merged[f'4h_ma_{teeth}']) & 
    (df_merged[f'4h_ma_{teeth}'] > df_merged[f'4h_ma_{jaw}']) &
    (df_merged[f'4h_ma_{jaw}_delta'] > 0) &
    (df_merged[f'1h_ma_{lips}'] > df_merged[f'1h_ma_{teeth}']) & 
    (df_merged[f'1h_ma_{teeth}'] > df_merged[f'1h_ma_{jaw}']) &
    (df_merged[f'1h_ma_{jaw}_delta'] > 0) &
    (df_merged[f'1h_ma_{teeth}_delta'] > 0) &
    (df_merged[f'1h_ma_{lips}_delta'] > 0) &
    (df_merged[f'15m_ma_{lips}'] > df_merged[f'15m_ma_{teeth}']) &
    (df_merged[f'15m_ma_{jaw}_delta'] > 0) &
    (df_merged[f'15m_ma_{teeth}_delta'] > 0) &
    (df_merged[f'15m_ma_{lips}_delta'] > 0)
]

# Сохраняем отфильтрованный результат в новый Excel файл
save_df_to_excel(filtered_df, "filtered_data")

# Поиск сделок в ранее отфильтрованных данных по Варианту 1

# Assuming df_filtered_data is your input DataFrame
df = filtered_df.copy()
df['time_open'] = pd.to_datetime(df['time_open'])

# Step 1: Identify continuous intervals
df['time_diff'] = df['time_open'].diff()
df['new_interval'] = df['time_diff'] > dt.timedelta(minutes=15)
df['interval_id'] = df['new_interval'].cumsum()

# Step 2: Create the new DataFrame with required information
result = []

for interval_id, group in df.groupby('interval_id'):
    start = group['time_open'].iloc[0]
    end = group['time_open'].iloc[-1]
    open_price = group['15m_open'].iloc[0]
    close_price = group['15m_close'].iloc[-1]
    max_price = max(group['15m_open'].max(), group['15m_close'].max())
    min_price = min(group['15m_open'].min(), group['15m_close'].min())
    
    result.append({
        'start': start,
        'end': end,
        'open': open_price,
        'close': close_price,
        'max': max_price,
        'min': min_price
    })

df_intervals = pd.DataFrame(result)
df_intervals.info


NameError: name 'df_merged' is not defined


Интервалы данных сохранены в файл: ~/Downloads/intervals__2025_01_01_19_31_57.571269.xlsx
